<a href="https://colab.research.google.com/github/OmarMlaeb/AAI612_Malaeb/blob/master/Week%204/Notebook4.4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# AAI612: Deep Learning & its Applications

*Notebook 4.3: Graded Assignment: Mini Project I*

<a href="https://colab.research.google.com/github/OmarMlaeb/AAI612_Malaeb/blob/master/Week%204/Notebook4.4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assessment

In this assessment, you will train a new model that is able to recognize fresh and rotten fruit. You will need to get the model to a validation accuracy of `92%` in order to pass the assessment, though we challenge you to do even better if you can. You will have the use the skills that you learned in the previous exercises. Specifically, we suggest using some combination of transfer learning, data augmentation, and fine tuning.

## The Dataset

In this exercise, you will train a model to recognize fresh and rotten fruits. Download the dataset from [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification). The dataset structure is in the `data/fruits` folder. There are 6 categories of fruits: fresh apples, fresh oranges, fresh bananas, rotten apples, rotten oranges, and rotten bananas. This will mean that your model will require an output layer of 6 neurons to do the categorization successfully. You'll also need to compile the model with `categorical_crossentropy`, as we have more than two categories.

![image.png](attachment:4c8c02c9-0cbe-4048-8d01-cdd5e3cf3fe6.png)<img src="./images/fruits.png" style="width: 600px;">

## Load ImageNet Base Model

Start with a model pretrained on `ImageNet`. Load the model with the correct weights, set an input shape, and choose to remove the last layers of the model. Remember that images have three dimensions: a height, and width, and a number of channels. Because these pictures are in color, there will be three channels for red, green, and blue. We've filled in the input shape for you. This cannot be changed or the assessment will fail. If you need a reference for setting up the pretrained model, please take a look at [Notebook 4.2](https://github.com/harmanani/AAI612/blob/main/Week4/Notebook%204.2.ipynb) where we implemented transfer learning.

In [ ]:
import ssl
from tensorflow import keras

ssl._create_default_https_context = ssl._create_unverified_context

# loading VGG16 without the top layers
base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False) # fixed

## Freeze Base Model

Next, we suggest freezing the base model. This is done so that all the learning from the ImageNet dataset does not get destroyed in the initial training.

In [ ]:
# Freeze base model
base_model.trainable = False # fixed

## Add Layers to Model

Now it's time to add layers to the pretrained model. Pay close attention to the last dense layer and make sure it has the correct number of neurons to classify the different types of fruit.  You may add more layers than specified below.

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten

# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3)) # fixed

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = GlobalAveragePooling2D()(x)  # used global average pooling instead of flatten
x = Dense(256, activation='relu')(x)  # added a dense hidden layer
x = Dropout(0.5)(x)  # dropout to prevent overfitting

# Add final dense layer
outputs = keras.layers.Dense(6, activation='softmax')(x) # 6 output neurons (one per class)

# Combine inputs and outputs to create model
model = keras.Model(inputs=inputs, outputs=outputs) # fixed

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 6)                 1542      
                                                             

## Compile Model

Now it's time to compile the model with loss and metrics options. Remember that we're training on a number of different categories, rather than a binary classification problem.

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

## Augment the Data

If you'd like, try to augment the data to improve the dataset. There is also documentation for the [Keras ImageDataGenerator class](https://keras.io/api/preprocessing/image/#imagedatagenerator-class). This step is optional, but it may be helpful to get to 92% accuracy.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # splitted dataset into 80% train, 20% validation
)

## Load Dataset

Now it's time to load the train and validation datasets. Pick the right folders, as well as the right `target_size` of the images (it needs to match the height and width input of the model you've created).

In [ ]:
# Load and iterate training dataset
train_it = datagen.flow_from_directory(
    'data/fruits',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode="categorical",
    subset="training"
) # fixed

# Load and iterate validation dataset
valid_it = datagen.flow_from_directory(
    'data/fruits',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode="categorical",
    subset="validation"
) # fixed

Found 8723 images belonging to 6 classes.
Found 2178 images belonging to 6 classes.


## Train the Model

Time to train the model! Pass the `train` and `valid` iterators into the `fit` function, as well as setting your desired number of epochs.

In [ ]:
model.fit(
    train_it,
    validation_data=valid_it,
    steps_per_epoch=train_it.samples//train_it.batch_size,
    validation_steps=valid_it.samples//valid_it.batch_size,
    epochs=20
) # fixed

Epoch 1/20
272/272 [==============================] - 123s 382ms/step - loss: 0.9790 - accuracy: 0.6392 - val_loss: 0.5255 - val_accuracy: 0.8364
Epoch 2/20
272/272 [==============================] - 64s 235ms/step - loss: 0.5327 - accuracy: 0.8095 - val_loss: 0.3639 - val_accuracy: 0.8690
Epoch 3/20
272/272 [==============================] - 71s 261ms/step - loss: 0.4252 - accuracy: 0.8434 - val_loss: 0.2972 - val_accuracy: 0.8998
Epoch 4/20
272/272 [==============================] - 115s 424ms/step - loss: 0.3681 - accuracy: 0.8654 - val_loss: 0.2693 - val_accuracy: 0.9058
Epoch 5/20
272/272 [==============================] - 115s 424ms/step - loss: 0.3308 - accuracy: 0.8783 - val_loss: 0.2317 - val_accuracy: 0.9145
Epoch 6/20
272/272 [==============================] - 120s 440ms/step - loss: 0.3017 - accuracy: 0.8877 - val_loss: 0.1898 - val_accuracy: 0.9375
Epoch 7/20
272/272 [==============================] - 115s 421ms/step - loss: 0.2739 - accuracy: 0.9000 - val_loss: 0.1984 - v

## Unfreeze Model for Fine Tuning

If you have reached 92% validation accuracy already, this next step is optional. If not, we suggest fine tuning the model with a very low learning rate.

In [ ]:
# Unfreeze the base model
base_model.trainable = True # fixed

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy']) # fixed

In [ ]:
model.fit(
    train_it,
    validation_data=valid_it,
    steps_per_epoch=train_it.samples//train_it.batch_size,
    validation_steps=valid_it.samples//valid_it.batch_size,
    epochs=10
)

Epoch 1/10
272/272 [==============================] - 131s 439ms/step - loss: 0.2019 - accuracy: 0.9406 - val_loss: 0.0486 - val_accuracy: 0.9839
Epoch 2/10
272/272 [==============================] - 115s 424ms/step - loss: 0.0773 - accuracy: 0.9733 - val_loss: 0.0580 - val_accuracy: 0.9789
Epoch 3/10
272/272 [==============================] - 116s 425ms/step - loss: 0.0451 - accuracy: 0.9864 - val_loss: 0.0554 - val_accuracy: 0.9825
Epoch 4/10
272/272 [==============================] - 116s 424ms/step - loss: 0.0302 - accuracy: 0.9895 - val_loss: 0.0105 - val_accuracy: 0.9982
Epoch 5/10
272/272 [==============================] - 116s 424ms/step - loss: 0.0280 - accuracy: 0.9921 - val_loss: 0.0022 - val_accuracy: 0.9991
Epoch 6/10
272/272 [==============================] - 115s 423ms/step - loss: 0.0197 - accuracy: 0.9939 - val_loss: 0.0036 - val_accuracy: 0.9991
Epoch 7/10
272/272 [==============================] - 115s 423ms/step - loss: 0.0171 - accuracy: 0.9952 - val_loss: 0.0011 -

## Evaluate the Model

Hopefully, you now have a model that has a validation accuracy of 92% or higher. If not, you may want to go back and either run more epochs of training, or adjust your data augmentation.

## Discussion

In this project, we used transfer learning with the VGG16 model to classify images into six different categories. At first, we loaded the pre-trained VGG16 model without its fully connected layers and kept its ImageNet weights to take advantage of its previously learned features. To prevent overfitting, we froze the model’s parameters and added a Global Average Pooling layer instead of Flatten, followed by dense layers with ReLU activation and Dropout for regularization.

To make the model more robust, we applied data augmentation techniques like rotation, zoom, and flipping using ImageDataGenerator. This helped improve generalization and prevent overfitting. The model was then compiled and trained for 20 epochs.

Since the initial training results showed good results, we fine-tuned the model by unfreezing the base layers and training it again with a lower learning rate using the RMSprop optimizer. This step allowed the model to refine its learned features, leading to a significant increase in accuracy. By balancing feature extraction and fine-tuning, we achieved near-perfect validation accuracy, making this approach well-suited for small to medium-sized datasets.

#### Phase 1: Initial Model Training

During the initial training phase, the model showed consistent improvement in both training and validation accuracy. In which the first epoch started with 63.92% accuracy, and after 20 epochs, it reached 93.20% training accuracy and 95.04% validation accuracy. The model's loss gradually decreased, showing that it was learning effectively. However the validation accuracy fluctuated slightly in later epochs, indicating the need for fine-tuning.

#### Phase 2: Fine-Tuning with Unfreezing and Lower Learning Rate

To optimize performance I unfrozed the model layers to allow fine-tuning of deeper representations and lowered the learning rate to stabilize the training process. Showing that the results were significantly improved, the model reached 99.70% training accuracy and 99.86% validation accuracy after 10 epochs. The validation accuracy hit 100% during some epochs and the validation loss became extremely low, reaching near zero in later epochs.


As we can see Fine-tuning increased performance, the accuracy jumped from 93.20% to 99.70%, indicating that unfreezing layers allowed the model to extract deeper features from the data. moreover, lowering the learning rate stabilized training in which it avoided abrupt weight updates and reduced the risk of overfitting. While the model achieved near-perfect accuracy, the very low validation loss suggests it may have memorized the training data.